# Jupyter Class Helper
---
These files are used to configure and organize the website's contents.

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [1]:
# Always run this before any of the following cells
import pandas as pd
import numpy as np
import csv
import logging
import subprocess
import yaml
import builder as bd
from pathlib import Path
import gdown
base_path=Path('..')
config_path = base_path / 'config'
cf=bd.load_yaml_file(config_path / 'config.yml')
excel_file= config_path / cf['excel_file']
class_path= base_path / cf['class']
content_path = class_path / 'content'

In [ ]:

https://docs.google.com/spreadsheets/d/1Y0dBlWdaTU0LkXrFutnDpjosdwnHqE2D/edit?usp=sharing&ouid=102358270837930135419&rtpof=true&sd=true
#https://docs.google.com/spreadsheets/d/1Y0dBlWdaTU0LkXrFutnDpjosdwnHqE2D/edit?usp=sharing&ouid=102358270837930135419&rtpof=true&sd=true

In [2]:
#First download the google shared version of the files
url = 'https://drive.google.com/uc?id='+cf['google_fileid']
gdown.download(url, str(config_path / cf['excel_file']), quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1E0Y0lC-q6uYh6So-VEVT60k7_KJFn_xH
To: /Users/jasonkuruzovich/GitHub/website_fall_2022/config/class.xlsx
100%|██████████| 141k/141k [00:00<00:00, 2.10MB/s]


'../config/class.xlsx'

In [3]:
# These load configuration from the excel files 
config = bd.load_yaml_file(class_path / '_config.yml') # Load the file.
toc = bd.load_yaml_file(class_path / '_toc.yml') # Load the file.
config_xl= pd.read_excel(excel_file, sheet_name = '_config_yml', header=None, index_col=None)
schedule= pd.read_excel(excel_file, sheet_name = 'Schedule',  index_col=None)
content={}
content['before_class']= pd.read_excel(excel_file, sheet_name = 'Before',  index_col=None)
content['in_class']= pd.read_excel(excel_file, sheet_name = 'During',  index_col=None)
#content['Assignment']= pd.read_excel(excel_file, sheet_name = 'Assignments',  index_col=None)


In [4]:
content['in_class']

,Session,Content,Location,Type
0,1,Powerpoint,see box.,NaN


In [5]:
#Create the syllabus link.
#The second value of the index postion of the syllabus on the before class content.
#bd.create_syllabus(content['Before Class'],0,cf['syllabus_message'],content_path / 'syllabus.md', config['repository']['url'])

In [6]:
#Fix in case individual tries to publish where session is NA. This isn't allowed. 
schedule.loc[schedule['Session'].isna(),'Publish']=0. 

In [7]:
#Generate Links from the schedule to the sessions and within the other tables. 
schedule.loc[schedule['Publish']==1,'Location']=schedule.loc[schedule['Publish']==1,'Session'].apply(lambda x: '../sessions/session'+str(int(x)))
schedule.loc[schedule['Publish']==1,'Type']='Markdown'
schedule=bd.link_generator(schedule, 'Summary',config['repository']['url'],'Link')
#content['Assignment']=bd.link_generator(content['Assignment'], 'Assignment',config['repository']['url'],'Starter')
content['before_class']=bd.link_generator(content['before_class'], 'Content',config['repository']['url'],'Link')
content['in_class']=bd.link_generator(content['in_class'], 'Content',config['repository']['url'],'Link')

In [8]:
#Get the in class activities and prepare and output a markdown file. 
schedule_ic=schedule.merge(content['in_class'], left_on='Session', right_on='Session', how='left')


In [9]:
schedule_ic= schedule_ic.loc[schedule_ic['Content'].notnull(),['Week', 'Session', 'Date', 'Content']]


In [10]:
type(schedule_ic['Date'][0])

datetime.datetime

In [11]:
schedule_ic=bd.pandas_to_md(schedule_ic, content_path / 'in_class.md', 'in_class', \
        include = ['Week', 'Session', 'Date', 'Content'], header=cf['in_class_header'])

Converting datetime to 
Outputting file: ../site/content/in_class.md


In [12]:
#Get the before class activities and prepare and output a markdown file. 
schedule_bc=schedule.merge(content['before_class'], left_on='Session', right_on='Session', how='left')
schedule_bc= schedule_bc.loc[schedule_bc['Content'].notnull(),['Week', 'Session', 'Date', 'Content']]
schedule_bc=bd.pandas_to_md(schedule_bc, content_path / 'preparation.md', 'Before Class', \
                             include = ['Week', 'Session', 'Date', 'Content'], header=cf['bc_class_header'])
#schedule=schedule.merge(content['Assignment'], left_on='Session', right_on='Session', how='left')



Converting datetime to 
Outputting file: ../site/content/preparation.md


#Get the assignments and prepare and output a markdown file. 
assignments_new = schedule.loc[schedule['Assignment'].notnull(),['Week', 'Session', 'Date', 'Assignment', 'Due']]
if len(assignments_new)>0:
    assignments_new=bd.pandas_to_md(assignments_new, content_path / 'assignments.md', 'Assignments', \
                            include = ['Week', 'Session', 'Date', 'Assignment', 'Due'],header=cf['assignments_header'])

In [13]:
#Output the schedule to markdown.
schedule=bd.pandas_to_md(schedule, content_path / 'schedule.md', 'Schedule', \
                             include = ['Week', 'Session', 'Date', 'Day', 'Topic', 'Summary'],header=cf['schedule_header'])

Converting datetime to 
Outputting file: ../site/content/schedule.md


In [14]:
#Generate Session Files
#generate_sessions(config, toc, toc_part, schedule, path, content, keys):
bd.generate_sessions(config=config, toc=toc, schedule=schedule, path=class_path / 'sessions', content=content, keys=['before_class', 'in_class'])

Outputting  session1.md
Outputting  session2.md
Outputting  session3.md
Outputting  session4.md
Outputting  session5.md
Outputting  session6.md
Outputting  session7.md
Outputting  session8.md
Outputting  session9.md
Outputting  session10.md


In [15]:
#Update the sessions to the yaml file.  Other updates to notebooks need to be done manually.
#bd.update_yaml_file(class_path / '_toc.yml', toc)

In [16]:
#TBD Make it so that notebooks will show up in _toc.yml. 